In [1]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3
)
from yolov3_tf2.dataset import transform_images
from yolov3_tf2.utils import draw_outputs    

from IPython.display import clear_output

clear_output()

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

mtl_model = YoloV3(416,
                    channels=3,
                    classes=1, 
                    disaster_classes = 7,
                    yolo_iou_threshold=0.5,
                    yolo_score_threshold=0.4,
                    auxiliary=True,
                    training=False
                  )

mtl_model.load_weights('mtl_model.h5')
clear_output()

2022-07-01 18:19:38.011756: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-01 18:19:38.023070: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-01 18:19:38.024552: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


AttributeError: 'str' object has no attribute 'decode'

In [4]:
mtl_model.outputs

[<KerasTensor: shape=(1, None, 4) dtype=float32 (created by layer 'yolo_nms')>,
 <KerasTensor: shape=(1, None) dtype=float32 (created by layer 'yolo_nms')>,
 <KerasTensor: shape=(1, None) dtype=int64 (created by layer 'yolo_nms')>,
 <KerasTensor: shape=(1,) dtype=int32 inferred_value=[None] (created by layer 'yolo_nms')>,
 <KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'disaster_head')>]

## Method 1: Convert to SavedModel Dir

In [5]:
mtl_model.save("mtl_model1")

2022-07-01 18:20:39.141903: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mtl_model1/assets


## Method 2: Convert to pb file
https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [6]:
#path of the directory where you want to save your model
frozen_out_path = 'frozen_graph'
# name of the .pb file
frozen_graph_filename = "mtl_model"

model = YoloV3(416,
                    channels=3,
                    classes=1, 
                    disaster_classes = 7,
                    yolo_iou_threshold=0.5,
                    yolo_score_threshold=0.4,
                    auxiliary=True,
                    training=False
                  )

model.load_weights('mtl_model.h5')

In [7]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen graph def
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

2022-05-31 22:27:39.181883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 22:27:39.182187: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-05-31 22:27:39.182251: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-31 22:27:39.182733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 22:27:39.182929: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 Super computeCapability: 7.5
coreClock: 1.38GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-05-31 22:27

------------------------------------------------------------
Frozen model layers: 
x
yolov3/yolo_nms/sub/x
yolov3/yolo_output_0/lambda_3/Reshape/shape/0
yolov3/yolo_darknet/conv2d_85/Conv2D/ReadVariableOp/resource
yolov3/yolo_darknet/conv2d_85/Conv2D/ReadVariableOp
yolov3/yolo_darknet/conv2d_85/Conv2D
yolov3/yolo_darknet/batch_normalization_85/ReadVariableOp/resource
yolov3/yolo_darknet/batch_normalization_85/ReadVariableOp
yolov3/yolo_darknet/batch_normalization_85/ReadVariableOp_1/resource
yolov3/yolo_darknet/batch_normalization_85/ReadVariableOp_1
yolov3/yolo_darknet/batch_normalization_85/FusedBatchNormV3/ReadVariableOp/resource
yolov3/yolo_darknet/batch_normalization_85/FusedBatchNormV3/ReadVariableOp
yolov3/yolo_darknet/batch_normalization_85/FusedBatchNormV3/ReadVariableOp_1/resource
yolov3/yolo_darknet/batch_normalization_85/FusedBatchNormV3/ReadVariableOp_1
yolov3/yolo_darknet/batch_normalization_85/FusedBatchNormV3
yolov3/yolo_darknet/leaky_re_lu_72/LeakyRelu
yolov3/yolo_dark

In [8]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

'frozen_graph/mtl_model_rpi.pbtxt'

In [9]:
def toy_dataset():
    inputs = tf.range(10.)[:, None]
    labels = inputs * 5. + tf.range(5.)[None, :]
    return tf.data.Dataset.from_tensor_slices(
        dict(x=inputs, y=labels)).repeat().batch(2)

opt = tf.keras.optimizers.Adam(0.01)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [ ]:
#######